In [3]:
import sys
sys.path.append('../src')
from pipeline import GCLPipeline

import os.path as osp
import json

import torch
import torch_geometric.transforms as T

from torch.optim import Adam

from tqdm import tqdm 

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

DATA_PATH = "../data"
BENCHMARK_PATH = "../benchmark"

In [4]:
STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark.json")))
CONFIG = STRATEGIES["config"]
TRY_GPU = CONFIG["try_gpu"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() and TRY_GPU else "cpu")

In [5]:
for strategy in STRATEGIES["strategies"]:

    print("\n")

    # Initialization
    pipeline = GCLPipeline.from_strategy(strategy, DEVICE)

    # Data
    dataset, num_features = GCLPipeline.init_dataset(
        strategy["dataset"], DATA_PATH, T.NormalizeFeatures(), strategy["batch_size"]
    )
    
    # Encoder
    encoder_params = strategy["encoder_params"]
    encoder_params["input_dim"] = num_features
    encoder_model = pipeline.init_encoder(encoder_params, DEVICE)

    # Training
    train_params = strategy["train_params"]
    lr = train_params["learning_rate"]
    epochs = train_params["epochs"]

    optimizer = Adam(encoder_model.parameters(), lr=lr)
    losses = []

    with tqdm(total=epochs, desc='(T)') as pbar:
        for epoch in tqdm(range(1, epochs+1)):
            loss = pipeline.train_epoch(
                encoder_model, dataset, optimizer, DEVICE
            )
            pbar.set_postfix({'loss': loss})

            pbar.update()

    test_result = pipeline.test(encoder_model, dataset, DEVICE)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')




##### InfoGraph #####
Dataset initialization
	 # features: 18
Encoder initialization
	 input dim: 18
	 hidden dim: 32
	 # layers: 2
	 projection dim: None
	 activation: <class 'torch.nn.modules.activation.ReLU'>


(T): 100%|██████████| 100/100 [00:06<00:00, 16.27it/s, loss=-2.19]
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number 

(E): Best test F1Mi=0.6571, F1Ma=0.6500


##### TransductiveDGI #####
Dataset initialization
	 # features: 1433
Encoder initialization
	 input dim: 1433
	 hidden dim: 512
	 # layers: 2
	 projection dim: None
	 activation: None


(LR): 100%|██████████| 5000/5000 [00:07<00:00, best test F1Mi=0.857, F1Ma=0.815]


(E): Best test F1Mi=0.8566, F1Ma=0.8150


##### GRACE #####
Dataset initialization
	 # features: 1433
Encoder initialization
	 input dim: 1433
	 hidden dim: 32
	 # layers: 2
	 projection dim: 32
	 activation: <class 'torch.nn.modules.activation.ReLU'>


(T):   0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
(LR): 100%|██████████| 5000/5000 [00:07<00:00, best test F1Mi=0.743, F1Ma=0.714]

(E): Best test F1Mi=0.7426, F1Ma=0.7143


In [4]:
del STRATEGIES

In [13]:
testlist = [None, None]
testlist= ["a", "b"]
if not testlist:
    print(1)


In [12]:
from multipledispatch import dispatch

@dispatch(float, float, float)
def product(a, b, c):
    return a*b*c

@dispatch(int, int)
def product(a, b):
    return a*b

product(2.0, 3.0, 4.0)

24.0

In [5]:

import torch. nn as nn

activation_func = getattr(nn, "ReLU", None)()
activation_func
    # if activation_func is not None:
    #     return activation_func()
    # else:
    #     raise ValueError(f"Unknown activation function: {name}")

ReLU()